In [24]:
import pandas as pd
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/nikolai/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
df = pd.read_csv('./data/tweets.csv')

In [19]:
df = df['text']
df.to_csv(r'./data/text_only.txt', header=None, index=None, sep=' ', mode='a')

In [10]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

In [20]:
reader = PlaintextCorpusReader('./data/', '.*\.txt')

In [27]:
reader.sents('text_only.txt')[1]

['"',
 'I',
 'was',
 'thrilled',
 'to',
 'be',
 'back',
 'in',
 'the',
 'Great',
 'city',
 'of',
 'Charlotte',
 ',',
 'North',
 'Carolina',
 'with',
 'thousands',
 'of',
 'hardworking',
 'American',
 'Patriots',
 'who',
 'love',
 'our',
 'Country',
 ',',
 'cherish',
 'our',
 'values',
 ',',
 'respect',
 'our',
 'laws',
 ',',
 'and',
 'always',
 'put',
 'AMERICA',
 'FIRST',
 '!']